In [5]:
# load in csvOut.csv data (holds image name & classification)
import csv
import os #to get all picture in directory
from PIL import Image #to open images
import numpy as np #load in numpy module
import sys

# loads in full csvOut data (name & classification)
with open('..//annotations_handheld.csv', newline='') as csvfile:
    cls_full = list(csv.reader(csvfile))

In [10]:
# Get the list of all files in directory
path = "..//images_handheld_resized"
dir_list = os.listdir(path)

# Initialize list of 'keys'
keys = ["regular", "double", "half"]

# Initialize lists to hold bitmap information for each image
bitmap = {}
for key in keys:
    bitmap[key] = []

# list of classifications
cls = []

# loop through files and get bit map for each (save as object where filename => bitmap for r,g,b)
for index, file in enumerate(cls_full):
    
    # stop at 100 files for time being
    if index == 100:
        break
    
    # push classification to list
    cls.append(file[1])
    
    # method found https://stackoverflow.com/questions/46385999/transform-an-image-to-a-bitmap
    img = Image.open(path + "\\" + file[0])
    
    # resize image
    #small_img = img.resize((600, 400))
    #smaller_img = img.resize((300, 200))
    smallest_img = img.resize((120, 80))
    
    # merge channels in new order
    bitmap['regular'].append(np.array(smallest_img).reshape(-1))
    bitmap['double'].append(np.array(smallest_img).reshape(-1))
    bitmap['half'].append(np.array(smallest_img).reshape(-1))

In [11]:
# convert bitmaps to indexed np array
bitmap_arr = {}
for key in keys:
    bitmap_arr[key] = np.array(bitmap[key])

# convert classifications to numpy array
cls_np = np.array(cls)

In [12]:
# adjust by some constant
bitmap_arr['double'] = bitmap_arr['double'] * 2
bitmap_arr['half'] = bitmap_arr['half'] / 2

In [20]:
bitmap_arr['regular']
#bitmap_arr['double']
#bitmap_arr['half']

array([[130, 226,  54, ..., 130, 120, 108],
       [ 98, 164, 110, ..., 130, 120, 110],
       [150, 218, 154, ..., 202,  70, 104],
       ...,
       [118, 202,  48, ..., 140, 170, 140],
       [148, 138, 112, ...,  70,  62,  58],
       [188, 204, 190, ..., 130, 120, 108]], dtype=uint8)

In [21]:
# Training model (load necessary libraries)
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Initialize dictionary to hold training & test data
X_train = {}
X_test = {}
y_train = {}
y_test = {}

# loop through keys split data into training and testing sets
for key in keys: 
    X_train[key], X_test[key], y_train[key], y_test[key] = train_test_split(bitmap_arr[key], cls_np, test_size=0.2, random_state = 0)

In [22]:
# set dictionary to hold predictions
y_pred = {}

# loop through keys, create SVM model and get classification prediction
for key in keys: 
    model = svm.SVC(kernel="poly", C=0.1, degree=2, coef0=0, gamma="scale")
    model.fit(X_train[key], y_train[key])
    y_pred[key] = model.predict(X_test[key])

In [23]:
# check if metamorphic relations hold
c1 = np.array_equal(y_pred['regular'], y_pred['double'])
c2 = np.array_equal(y_pred['double'], y_pred['half'])

if c1 & c2:
    print("Metamorphic relation holds.")
else:
    print("Metamorphic relation does not hold.")

Metamorphic relation does not hold.


In [24]:
y_pred['regular']

array(['B', 'B', 'B', 'B', 'B', 'H', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'H', 'H', 'B', 'B'], dtype='<U1')

In [25]:
y_pred['double']

array(['B', 'B', 'B', 'B', 'B', 'H', 'B', 'B', 'B', 'H', 'B', 'B', 'B',
       'B', 'B', 'B', 'H', 'B', 'B', 'B'], dtype='<U1')

In [26]:
y_pred['half']

array(['B', 'B', 'B', 'B', 'B', 'H', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'H', 'H', 'B', 'B'], dtype='<U1')